In [1]:
pip install transformers torch pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 620.7 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 5.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.7/401.7 kB 6.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.9/415.9 kB 2.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 6.5 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


##### Vectorizing Data Using BERT-Based Model

In [2]:
import json
import pandas as pd
import torch
from transformers import BertTokenizer, BertModel

def bert_vectorized(input_file_path, output_file_path):
    # Load the tokenized text from the JSON file
    with open(input_file_path, 'r') as file:
        tokenized_texts = json.load(file)
    
    # Initialize BERT model and tokenizer
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model = BertModel.from_pretrained('bert-base-uncased')

    # Ensure the model is in evaluation mode
    model.eval()

    # Function to get BERT embeddings
    def get_bert_embedding(tokens):
        # Tokenize and create tensor
        inputs = tokenizer(tokens, return_tensors='pt', is_split_into_words=True, padding=True, truncation=True)
        with torch.no_grad():
            outputs = model(**inputs)
        # Get the embeddings from the last hidden state
        embeddings = outputs.last_hidden_state
        # Take the mean of the token embeddings to get a sentence-level embedding
        sentence_embedding = torch.mean(embeddings, dim=1)
        return sentence_embedding.squeeze().numpy()

    # Process each tokenized text and get embeddings
    embeddings = []
    for tokens in tokenized_texts:
        embedding = get_bert_embedding(tokens)
        embeddings.append(embedding)

    # Convert embeddings to DataFrame and save as CSV
    df = pd.DataFrame(embeddings)
    df.to_csv(output_file_path, index=False)

# Example usage
input_file_path = 'tokenized_300.json'
output_file_path = 'vectorized_bert_300.csv'
bert_vectorized(input_file_path, output_file_path)


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
